In [2]:
!pip install pykrige 

In [3]:
import sys, site
print("Python del kernel:", sys.executable)
print("site-packages:", site.getsitepackages())
!{sys.executable} -m pip install pykrige
import pykrige, os
print("PyKrige:", pykrige.__version__)
print("Ubicación:", os.path.abspath(pykrige.__file__))


Python del kernel: /home/thomas/anaconda3/bin/python
site-packages: ['/home/thomas/anaconda3/lib/python3.12/site-packages']
PyKrige: 1.7.3
Ubicación: /home/thomas/anaconda3/lib/python3.12/site-packages/pykrige/__init__.py


In [149]:
# -*- coding: utf-8 -*-
"""
Kriging de precipitación (con suavizado) + mapa de estaciones
- Paleta/escala estilo SMN
- Capa de aguas permanentes
- Logo en esquina superior derecha
- Sin puntos de estaciones sobre el mapa kriging
"""

import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import unary_union
from shapely.prepared import prep
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from matplotlib.colors import ListedColormap, BoundaryNorm
from pykrige.ok import OrdinaryKriging
from scipy.ndimage import gaussian_filter

# =======================
# PARÁMETROS EDITABLES
# =======================
CSV_ACUM = "/home/thomas/Documentos/GitHub/repo/Ploteo-Acumulados/datos/precipitacion_acumulado_2025-10-24_11-26.csv"
CSV_COORD = "/home/thomas/Documentos/GitHub/repo/Ploteo-Acumulados/datos/station_coordinates.csv"
SHP_DEPTO = "/home/thomas/Documentos/GitHub/repo/Ploteo-Acumulados/SHP/departamentos.shp"
PROVINCIA_FILTRO = "CORDOBA"
WATER_SHP = os.path.join(os.path.dirname(SHP_DEPTO), "areas_de_aguas_continentales_perenne.shp")

# RUTA DEL LOGO (ajusta esta ruta a tu logo)
LOGO_PATH = "Logo_ohmc"  # Cambia esto

UTM_EPSG = 32720           # UTM 20S
GRID_SIZE_M = 2000         # tamaño de celda (m)
SMOOTH_SIGMA_CELLS = 1.3   # sigma del suavizado gaussiano (en celdas)

# --- Escala SMN (0.1 a 300 mm) ---
BREAKS = np.array([0.0, 0.1, 1, 2, 5, 10, 15, 20, 30, 50, 75, 100, 125, 150, 200, 250, 300], dtype=float)
COLORS = [
    "#f7f7f7",  # 0.1–1   (muy claro)
    "#eaf2c2",  # 1–2     (verde muy claro)
    "#cde9a3",  # 2–5     (verde claro)
    "#9fd67a",  # 5–10    (verde)
    "#73c05e",  # 10–15   (verde medio)
    "#4caf50",  # 15–20   (verde más oscuro)
    "#87d3e6",  # 20–30   (celeste)
    "#5bbff2",  # 30–50   (celeste/azul claro)
    "#2791d2",  # 50–75   (azul)
    "#2b4da0",  # 75–100  (azul oscuro)
    "#3b2f85",  # 100–125 (índigo)
    "#6a2c91",  # 125–150 (violeta)
    "#8e2ca8",  # 150–200 (magenta-violeta)
    "#c02ab4",  # 200–250 (magenta)
    "#e065cf",  # 250–300 (magenta claro)
    "#7a0177",  # >=300   (púrpura fuerte; saturado)
]

PNG_KRIG   = "kriging_precipitacion.png"
PNG_PUNTOS = "ubicacion_estaciones.png"

# =======================
# UTILIDADES
# =======================
def norm_id(x: str) -> str:
    x = str(x).strip()
    if x.endswith(".0"): x = x[:-2]
    x = x.lstrip("0")
    return x or "0"

def fmt_time_col(series):
    try:
        dt = pd.to_datetime(series, errors="coerce")
        if dt.notna().any():
            # Formato sin año: solo mes-día hora:minuto
            return dt.min().strftime("%m-%d %H:%M"), dt.max().strftime("%m-%d %H:%M")
    except Exception:
        pass
    return "", ""

def ensure_crs_wgs84(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    if gdf.crs is None:
        gdf = gdf.set_crs(epsg=4326)
    return gdf

def pick_coord_columns(df: pd.DataFrame):
    cols = {c.lower(): c for c in df.columns}
    lon_cands = ["longitude","lon","long","lng","x","coord_x","longitude_x","longitude_y","lon_x","lon_y"]
    lat_cands = ["latitude","lat","y","coord_y","latitude_x","latitude_y","lat_x","lat_y"]
    def best(cands):
        found = []
        for c in cands:
            if c in cols:
                col = cols[c]
                nn = df[col].notna().sum()
                found.append((nn, col))
        return max(found)[1] if found else None
    return best(lon_cands), best(lat_cands)

# =======================
# 1) CARGA / UNIÓN / SANEOS
# =======================
df = pd.read_csv(CSV_ACUM)
df2 = pd.read_csv(CSV_COORD)
depto = ensure_crs_wgs84(gpd.read_file(SHP_DEPTO))

depto = depto.loc[depto["PROVINCIA"] == PROVINCIA_FILTRO].copy()
if depto.empty:
    raise RuntimeError(f"No se encontraron polígonos con PROVINCIA == '{PROVINCIA_FILTRO}'.")

df["station_code_norm"] = df["station_code"].apply(norm_id)
df2["original_id_norm"]  = df2["original_id"].apply(norm_id)

dfj = pd.merge(df, df2, left_on="station_code_norm", right_on="original_id_norm",
               how="left", suffixes=("", "_coord"))
dfj["accumulated_rain_mm"] = pd.to_numeric(dfj.get("accumulated_rain_mm"), errors="coerce")

lon_col, lat_col = pick_coord_columns(dfj)
if lon_col is None or lat_col is None:
    lon_df, lat_df = pick_coord_columns(df)
    lon_df2, lat_df2 = pick_coord_columns(df2)
    if lon_df and lat_df:
        dfj["__lon_tmp__"] = df[lon_df]; dfj["__lat_tmp__"] = df[lat_df]
        lon_col, lat_col = "__lon_tmp__", "__lat_tmp__"
    elif lon_df2 and lat_df2:
        dfj["__lon_tmp__"] = df2[lon_df2]; dfj["__lat_tmp__"] = df2[lat_df2]
        lon_col, lat_col = "__lon_tmp__", "__lat_tmp__"
    else:
        raise RuntimeError("No pude detectar columnas de coordenadas (lon/lat).")

dfj[lon_col] = pd.to_numeric(dfj[lon_col], errors="coerce")
dfj[lat_col] = pd.to_numeric(dfj[lat_col], errors="coerce")
dfj_ok = dfj.dropna(subset=[lon_col, lat_col, "accumulated_rain_mm"]).copy()
if dfj_ok.empty:
    raise RuntimeError(f"No hay filas con {lon_col}/{lat_col} y accumulated_rain_mm válidos.")

first_reading_str, last_reading_str = "", ""
if "first_reading" in dfj_ok.columns and "last_reading" in dfj_ok.columns:
    a, b = fmt_time_col(dfj_ok["first_reading"]); c, d = fmt_time_col(dfj_ok["last_reading"])
    first_reading_str = a or c or ""; last_reading_str  = d or b or ""

# =======================
# 2) GEOMETRÍA / PROYECCIONES
# =======================
gdf_pts = gpd.GeoDataFrame(dfj_ok,
    geometry=gpd.points_from_xy(dfj_ok[lon_col], dfj_ok[lat_col]), crs="EPSG:4326")
gdf_ciudades = gpd.GeoDataFrame(
    {"nombre": ["Córdoba","Villa María","Río Cuarto","Mina Clavero","V.Mackenna","Laboulaye","Cruz del Eje","Villa de María"],
     "geometry": gpd.points_from_xy([ -64.1914,-63.2438,-64.3497,-65.0044,-64.3907,-63.3885,-64.8076,-63.7234],
                                    [ -31.4193,-32.4465,-33.1230,-31.7296,-33.9183,-34.1276,-30.7209,-29.9043])},
    crs="EPSG:4326"
)

depto_utm       = depto.to_crs(epsg=UTM_EPSG)
gdf_pts_utm     = gdf_pts.to_crs(epsg=UTM_EPSG)
gdf_ciudades_utm= gdf_ciudades.to_crs(epsg=UTM_EPSG)

# Aguas permanentes
agua = ensure_crs_wgs84(gpd.read_file(WATER_SHP))
agua_utm = agua.to_crs(epsg=UTM_EPSG)
try:
    agua_utm = gpd.clip(agua_utm, depto_utm)
except Exception:
    agua_utm = gpd.overlay(agua_utm, depto_utm, how="intersection")

# =======================
# 3) CLIP ESTACIONES A PROVINCIA
# =======================
geom_union = unary_union(depto_utm.buffer(0).geometry.values)
if isinstance(geom_union, Polygon): geom_union = MultiPolygon([geom_union])
prep_geom = prep(geom_union)

minx, miny, maxx, maxy = depto_utm.total_bounds
pad = 10000
bbox_poly = Polygon([(minx-pad, miny-pad),(maxx+pad, miny-pad),(maxx+pad, maxy+pad),(minx-pad, maxy+pad)])
gdf_pts_utm = gdf_pts_utm[gdf_pts_utm.geometry.within(bbox_poly)].copy()
gdf_pts_utm = gdf_pts_utm[[prep_geom.covers(geom) for geom in gdf_pts_utm.geometry]]
if len(gdf_pts_utm) == 0:
    gdf_pts_utm = gdf_pts.to_crs(epsg=UTM_EPSG)
if len(gdf_pts_utm) < 3:
    raise RuntimeError(f"Se necesitan ≥3 estaciones para kriging. Quedaron: {len(gdf_pts_utm)}.")

# =======================
# 4) GRILLA + MÁSCARA
# =======================
minx, miny, maxx, maxy = depto_utm.total_bounds
gx = np.arange(minx, maxx + GRID_SIZE_M, GRID_SIZE_M)
gy = np.arange(miny, maxy + GRID_SIZE_M, GRID_SIZE_M)
GX, GY = np.meshgrid(gx, gy)
flat_pts = [Point(xy) for xy in zip(GX.ravel(), GY.ravel())]
mask_in = np.fromiter((prep_geom.covers(p) for p in flat_pts), count=len(flat_pts), dtype=bool).reshape(GX.shape)

# =======================
# 5) KRIGING + SUAVIZADO
# =======================
x = gdf_pts_utm.geometry.x.values
y = gdf_pts_utm.geometry.y.values
z = gdf_pts_utm["accumulated_rain_mm"].astype(float).values

sill = float(np.nanvar(z)); nugget = 0.05 * max(sill, 1e-6)
range_guess = float(max(maxx - minx, maxy - miny) / 3.0)
OK = OrdinaryKriging(x, y, z, variogram_model="spherical",
                     variogram_parameters={"sill": max(sill,1e-6),"range":max(range_guess,1e-6),"nugget":max(nugget,0.0)},
                     enable_plotting=False, coordinates_type="euclidean", verbose=False)

Z, _ = OK.execute("grid", gx, gy)
Z = np.array(Z)

# enmascarar fuera y valores mínimos
Z = np.where(mask_in, Z, np.nan)
Z = np.where((~np.isnan(Z)) & (Z < 0.1), np.nan, Z)

# --- SUAVIZADO GAUSSIANO respetando NaNs/máscara ---
valid = np.isfinite(Z).astype(float)
Z_filled = np.where(np.isfinite(Z), Z, 0.0)
Z_s  = gaussian_filter(Z_filled, sigma=SMOOTH_SIGMA_CELLS, mode="nearest")
W_s  = gaussian_filter(valid,   sigma=SMOOTH_SIGMA_CELLS, mode="nearest")
Z_sm = np.where(W_s > 0, Z_s / W_s, np.nan)

# saturar al rango de la escala SMN (evita valores >300 o <0.1)
Z_plot = np.clip(Z_sm, BREAKS[0], BREAKS[-1])

# =======================
# 6) MAPA KRIGING CON LOGO
# =======================
from numpy import ma as npma

# Colormap SMN y cortes
cmap = ListedColormap(COLORS)
norm = BoundaryNorm(BREAKS, ncolors=cmap.N, clip=True)

# Enmascarar TODO lo que esté fuera del polígono o sea NaN
Z_mask_poly = npma.array(Z_plot, mask=(~mask_in) | (~np.isfinite(Z_plot)))
cmap.set_bad(alpha=0.0)

# Crear figura con espacio superior para título y logo
fig = plt.figure(figsize=(16, 10), dpi=150)

# Crear grid de subplots personalizado
# [título | logo]
# [  mapa principal  ]
gs = fig.add_gridspec(2, 2, height_ratios=[0.12, 0.88], width_ratios=[0.75, 0.25],
                      hspace=0.02, wspace=0.05, 
                      left=0.60, right=0.92, top=0.96, bottom=0.08)

# Subplot para el mapa principal (ocupa ambas columnas en la fila inferior)
ax_map = fig.add_subplot(gs[1, :])

# Subplot para el título (arriba izquierda)
ax_title = fig.add_subplot(gs[0, 0])
ax_title.axis('off')

# Subplot para el logo (arriba derecha)
ax_logo = fig.add_subplot(gs[0, 1])
ax_logo.axis('off')

# Pintar el mapa
im = ax_map.imshow(
    Z_mask_poly,
    origin="lower",
    extent=(minx, maxx, miny, maxy),
    interpolation="nearest",
    cmap=cmap, norm=norm, zorder=1
)

# Aguas permanentes y bordes
agua_utm.plot(ax=ax_map, color="white", edgecolor="#6ea8d6", linewidth=0.6, zorder=3)
depto_utm.boundary.plot(ax=ax_map, linewidth=0.6, edgecolor="black", zorder=4)

# Ciudades
gdf_ciudades_utm.plot(ax=ax_map, color="black", markersize=20, zorder=5)
try:
    from adjustText import adjust_text
    texts = [ax_map.text(r.geometry.x, r.geometry.y, r["nombre"], fontsize=8, color="black", zorder=6)
             for _, r in gdf_ciudades_utm.iterrows()]
    if texts: adjust_text(texts, ax=ax_map, arrowprops=dict(arrowstyle="-", lw=0.3))
except Exception:
    for _, r in gdf_ciudades_utm.iterrows():
        ax_map.annotate(r["nombre"], (r.geometry.x, r.geometry.y), xytext=(5,5),
                        textcoords="offset points", fontsize=8, color="black")

# Estética del mapa
ax_map.set_aspect("equal")
ax_map.set_xticks([])
ax_map.set_yticks([])
for s in ax_map.spines.values(): 
    s.set_visible(False)

# Watermark
cx = (minx + maxx)/2
cy = (miny + maxy)/2
ax_map.text(cx, cy, "OHMC", fontsize=90, color="gray", alpha=0.25, rotation=45,
            ha="center", va="center", fontweight="bold", zorder=2)

# Colorbar 
cbar = fig.colorbar(im, ax=ax_map, fraction=0.045, pad=0.02, ticks=BREAKS[1:], format='%.0f')
cbar.ax.set_title("(mm)\n", fontsize=11, fontweight="bold")
cbar.ax.tick_params(labelsize=11)

# TÍTULO en el subplot superior izquierdo
# TÍTULO en el subplot superior izquierdo
title = "Precipitación Acumulada\núltimas 24 hs"

sub = f"{last_reading_str}" if (first_reading_str or last_reading_str) else ""

ax_title.text(0, 0.5, title + (f" ({sub})" if sub else ""), 
              fontsize=13, fontweight="bold", va='center', ha='left',
              transform=ax_title.transAxes)

# LOGO en el subplot superior derecho
if os.path.exists(LOGO_PATH):
    try:
        logo = Image.open(LOGO_PATH)
        ax_logo.imshow(logo)
    except Exception as e:
        print(f"No se pudo cargar el logo: {e}")
        ax_logo.text(0.1, 0.1, "LOGO", fontsize=20, ha='center', va='center',
                     bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))


# Caption (centrado respecto al mapa)
caption = ("Fuente: Estaciones meteorológicas de la provincia de Córdoba\n"
           "Elaborado con interpolación por Kriging (Ordinary Kriging)")
ax_map.text(0.5,-0.01, caption, ha="center", va="top", fontsize=9, color="gray",
            transform=ax_map.transAxes)

plt.savefig(PNG_KRIG, dpi=600, bbox_inches="tight", facecolor="white")
plt.close(fig)

print(f"Mapa guardado en: {PNG_KRIG}")

Mapa guardado en: kriging_precipitacion.png
